In [33]:
import pandas as pd
import akshare as ak

# 800自由现金流，季度更新
# https://www.csindex.com.cn/#/indices/family/detail?indexCode=932368
stocks = [
'000039',
'000426',
'000513',
'000568',
'000651',
'000708',
'000792',
'000807',
'000858',
'000933',
'002001',
'002056',
'002120',
'002352',
'002532',
'002714',
'300002',
'300073',
'300498',
'300724',
'300803',
'600066',
'600096',
'600219',
'600258',
'600295',
'600312',
'600352',
'600380',
'600482',
'600585',
'600873',
'600938',
'600968',
'601156',
'601168',
'601212',
'601225',
'601231',
'601600',
'601877',
'601880',
'601919',
'603129',
'603233',
'603816',
'603833',
'603885',
'603939',
'603993'
]
# 一次性获取全市场 A 股实时行情
spot_df = ak.stock_zh_a_spot_em()

  0%|          | 0/57 [00:00<?, ?it/s]

In [24]:
# 只看我们关心的列
spot_df = spot_df[["代码", "名称", "总市值"]]

# 过滤出你的股票
my_df = spot_df[spot_df["代码"].isin(stocks)]

In [26]:
def weekly_return(code):
    try:
        df = ak.stock_zh_a_hist(symbol=code, period="weekly", adjust="")
        if len(df) >= 2:
            last_week_close = df.iloc[-2]["收盘"]
            this_week_close = df.iloc[-1]["收盘"]
            return round((this_week_close / last_week_close - 1) * 100, 2)
    except Exception as e:
        print(f"{code} 获取失败: {e}")
    return None

# 添加周涨幅
my_df["周涨幅(%)"] = my_df["代码"].apply(weekly_return)

/tmp/ipykernel_91874/4148294583.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df["周涨幅(%)"] = my_df["代码"].apply(weekly_return)


In [32]:
filtered_df = my_df.sort_values(
    by=["总市值", "周涨幅(%)"],
    ascending=[True, False]    
)
filtered_df.head(10)

,代码,名称,总市值,周涨幅(%)
3219,600258,首旅酒店,1.579993e+10,1.14
3607,603233,大参林,1.934903e+10,-3.35
2162,600312,平高电气,2.154791e+10,-0.87
1330,603816,顾家家居,2.273842e+10,6.37
3505,601156,东航物流,2.292430e+10,0.00
1349,300073,当升科技,2.316514e+10,2.70
493,600380,健康元,2.343530e+10,4.98
3217,002120,韵达股份,2.371542e+10,-5.32
2626,601212,白银有色,2.502814e+10,0.60
45,300724,捷佳伟创,2.595334e+10,10.58
